# Exercise 4 : Regression Analysis using Text Data

Use regression to predict overall scores of reviews of Musical Instruments present in Amazon 

The dataset can be obtained from http://jmcauley.ucsd.edu/data/amazon/ http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Musical_Instruments_5.json.gz
Ups and downs: Modeling the visual evolution of fashion trends with one-class collaborative filtering R. He, J. McAuley WWW, 2016

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import re
import string
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import Counter
from pylab import *
import nltk
import warnings
warnings.filterwarnings('ignore')

In [2]:
review_data = pd.read_json('data_ch3/reviews_Musical_Instruments_5.json', lines=True)
review_data.head()

,asin,helpful,overall,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,1384719342,"[0, 0]",5,"Not much to write about here, but it does exac...","02 28, 2014",A2IBPI20UZIR0U,"cassandra tu ""Yeah, well, that's just like, u...",good,1393545600
1,1384719342,"[13, 14]",5,The product does exactly as it should and is q...,"03 16, 2013",A14VAT5EAX3D9S,Jake,Jake,1363392000
2,1384719342,"[1, 1]",5,The primary job of this device is to block the...,"08 28, 2013",A195EZSQDW3E21,"Rick Bennette ""Rick Bennette""",It Does The Job Well,1377648000
3,1384719342,"[0, 0]",5,Nice windscreen protects my MXL mic and preven...,"02 14, 2014",A2C00NNG1ZQQG2,"RustyBill ""Sunday Rocker""",GOOD WINDSCREEN FOR THE MONEY,1392336000
4,1384719342,"[0, 0]",5,This pop filter is great. It looks and perform...,"02 21, 2014",A94QU4C90B1AX,SEAN MASLANKA,No more pops when I record my vocals.,1392940800


In [3]:
review_data['overall'].value_counts()

5    6938
4    2084
3     772
2     250
1     217
Name: overall, dtype: int64

In [4]:
lemmatizer = WordNetLemmatizer()

In [5]:
review_data['cleaned_review_text'] = review_data['reviewText'].apply(\
lambda x : ' '.join([lemmatizer.lemmatize(word.lower()) \
    for word in word_tokenize(re.sub(r'([^\s\w]|_)+', ' ', str(x)))]))

In [6]:
review_data[['cleaned_review_text', 'reviewText', 'overall']].head()

,cleaned_review_text,reviewText,overall
0,not much to write about here but it doe exactl...,"Not much to write about here, but it does exac...",5
1,the product doe exactly a it should and is qui...,The product does exactly as it should and is q...,5
2,the primary job of this device is to block the...,The primary job of this device is to block the...,5
3,nice windscreen protects my mxl mic and preven...,Nice windscreen protects my MXL mic and preven...,5
4,this pop filter is great it look and performs ...,This pop filter is great. It looks and perform...,5


In [7]:
tfidf_model = TfidfVectorizer(max_features=500)
tfidf_df = pd.DataFrame(tfidf_model.fit_transform(review_data['cleaned_review_text']).todense())
tfidf_df.columns = sorted(tfidf_model.vocabulary_)
tfidf_df.head()

,10,100,12,20,34,able,about,accurate,acoustic,actually,...,won,work,worked,worth,would,wrong,year,yet,you,your
0,0.0,0.0,0.0,0.0,0.0,0.000000,0.159684,0.0,0.0,0.0,...,0.0,0.134327,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000
1,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.085436,0.0,0.0,0.000000,0.0,0.0,0.0,0.067074,0.000000
2,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.115312,0.0,0.0,0.0,0.079880,0.111989
3,0.0,0.0,0.0,0.0,0.0,0.339573,0.000000,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000
4,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.303608,0.000000


## Linear Rergression

In [9]:
from sklearn.linear_model import LinearRegression
linreg = LinearRegression()
linreg.fit(tfidf_df,review_data['overall'])

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [13]:
linreg.coef_

array([-1.93271993e-01,  5.65226131e-01,  5.63243687e-01, -1.84418658e-01,
       -6.32257431e-02,  3.05320627e-01,  4.95264614e-01,  5.21333693e-01,
        2.65736989e-01,  4.00058256e-01,  5.64020424e-01,  7.56022958e-01,
        1.00174846e-02, -3.06429115e-01, -3.12104234e-01,  3.38294736e-01,
       -6.05747380e-01, -1.04123996e-01,  5.58669738e-02, -1.13320890e-01,
        4.79471129e-01,  1.49528459e-01,  7.79094545e-01, -3.63399268e-01,
        1.25993539e-01, -6.29415062e-02,  4.94517373e-01, -3.34989132e-01,
        2.55374355e-01,  8.84676972e-02, -3.68013360e-01, -1.09910777e-01,
       -7.09777794e-03, -5.15547511e-02,  1.17415090e-01, -8.89213726e-02,
        1.06398798e+00, -1.19791236e+00, -1.14906460e+00,  1.55215016e-01,
       -5.05283241e-01,  2.43200389e-01,  8.56413437e-02, -3.74044994e-02,
       -7.31390217e-03,  9.63911076e-01, -7.82062558e-02,  1.50616236e-01,
       -9.35299622e-02,  1.87239631e-02,  9.34145997e-02,  1.18038260e+00,
       -3.79855115e-01,  

In [10]:
linreg.intercept_

4.218882428983381

In [14]:
linreg.predict(tfidf_df)

array([4.19200071, 4.25771652, 4.23084868, ..., 4.40384767, 4.49036403,
       4.14791976])

In [15]:
review_data['predicted_score_from_linear_regression'] = linreg.predict(tfidf_df)

In [18]:
review_data[['overall', 'predicted_score_from_linear_regression']].head(10)

,overall,predicted_score_from_linear_regression
0,5,4.192001
1,5,4.257717
2,5,4.230849
3,5,4.085927
4,5,4.851061
5,5,4.955069
6,5,4.446274
7,3,3.888593
8,5,4.941788
9,5,4.513824


Reference / Citation for the dataset: http://jmcauley.ucsd.edu/data/amazon/
http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Musical_Instruments_5.json.gz    
Ups and downs: Modeling the visual evolution of fashion trends with one-class collaborative filtering
R. He, J. McAuley WWW, 2016